References
- [Chris Mccormick finetuning BERT for SQUAD](https://colab.research.google.com/drive/16VjEulbATgok4mELTSaq7GTQdh3JGhGy#scrollTo=Xm1wTn09RAR7)
- [Discussion Regarding finetuning T5](https://github.com/huggingface/transformers/issues/4426) | [Exploring T5 by patil suraj](https://github.com/patil-suraj/exploring-T5)
    - [SQUAD QA finetuning for T5](https://colab.research.google.com/github/patil-suraj/exploring-T5/blob/master/T5_on_TPU.ipynb#scrollTo=KdmKlMkfcLa0)
    - [T5 finetuning for non extractive tasks](https://colab.research.google.com/drive/176NSaYjc2eeI-78oLH_F9-YV3po3qQQO?usp=sharing)
- [Google's T5 fine tuning example for QA](https://colab.research.google.com/github/google-research/text-to-text-transfer-transformer/blob/master/notebooks/t5-trivia.ipynb#scrollTo=6rU32DjyeLuL)

## Import from Baseline Code

In [1]:
# https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes, as dict.key_name, not as dict["key_name"] """
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


In [2]:
import logging
import os
import sys

from typing import List, Callable, NoReturn, NewType, Any
import dataclasses
from datasets import load_metric, load_from_disk, Dataset, DatasetDict

from transformers import AutoConfig, AutoModelForQuestionAnswering, AutoTokenizer

from transformers import (
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    TrainingArguments,
    set_seed,
)

from tokenizers import Tokenizer
from tokenizers.models import WordPiece

from baseline.utils_qa import postprocess_qa_predictions, check_no_error
from baseline.trainer_qa import QuestionAnsweringTrainer
from baseline.retrieval import SparseRetrieval

import yaml

# Read config.yaml file
with open("config.yaml") as infile:
    SAVED_CFG = yaml.load(infile, Loader=yaml.FullLoader)
    SAVED_CFG = dotdict(SAVED_CFG)

# arguments setting
data_args = dotdict(SAVED_CFG.data)
model_args = dotdict(SAVED_CFG.custom_model)
training_args = TrainingArguments(
    output_dir="./results",  # output directory
    save_total_limit=5,  # number of total save model.
    save_steps=model_args.save_steps,  # model saving step.
    num_train_epochs=model_args.num_train_epochs,  # total number of training epochs
    learning_rate=model_args.learning_rate,  # learning_rate
    per_device_train_batch_size=model_args.batch_size,  # batch size per device during training
    per_device_eval_batch_size=model_args.batch_size,  # batch size for evaluation
    warmup_steps=model_args.warmup_steps,  # number of warmup steps for learning rate scheduler
    weight_decay=model_args.weight_decay,  # strength of weight decay
    logging_dir="./logs",  # directory for storing logs
    logging_steps=100,  # log saving step.
    evaluation_strategy="steps",  # evaluation strategy to adopt during training
    # `no`: No evaluation during training.
    # `steps`: Evaluate every `eval_steps`.
    # `epoch`: Evaluate every end of epoch.
    eval_steps=500,  # evaluation step.
    load_best_model_at_end=True,
)


In [8]:
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW,AutoTokenizer, TrainingArguments, get_linear_schedule_with_warmup
from datasets import load_metric, load_from_disk, Dataset, DatasetDict
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)
from tqdm import tqdm
import random

tokenizer = AutoTokenizer.from_pretrained(model_args.model_name_or_path)
datasets = load_from_disk(data_args.dataset_name)
train_dataset_from_huggingface = datasets['train']
valid_dataset_from_huggingface = datasets['validation']

In [9]:
#train dataset, train dataloader
q_seqs = tokenizer(
    train_dataset_from_huggingface['question'], 
    padding="max_length", 
    truncation=True, 
    return_tensors='pt',
    return_token_type_ids=False,  # for RoBERTa
    )
p_seqs = tokenizer(
    train_dataset_from_huggingface['context'], 
    padding="max_length", 
    truncation=True, 
    return_tensors='pt',
    return_token_type_ids=False,  # for RoBERTa
    )
# print(q_seqs[0])
train_dataset = TensorDataset(
    p_seqs['input_ids'], 
    p_seqs['attention_mask'], 
    # p_seqs['token_type_ids'],
    q_seqs['input_ids'], 
    q_seqs['attention_mask'], 
    # q_seqs['token_type_ids']
    )
train_loader = DataLoader(train_dataset,batch_size=model_args.batch_size)

#valid dataset, valid dataloader
q_seqs = tokenizer(
    valid_dataset_from_huggingface['question'], 
    padding="max_length", 
    truncation=True, 
    return_tensors='pt',
    return_token_type_ids=False,  # for RoBERTa
    )
p_seqs = tokenizer(
    valid_dataset_from_huggingface['context'], 
    padding="max_length", 
    truncation=True, 
    return_tensors='pt',
    return_token_type_ids=False,  # for RoBERTa
    )
# print(q_seqs[0])
valid_dataset = TensorDataset(
    p_seqs['input_ids'], 
    p_seqs['attention_mask'], 
    # p_seqs['token_type_ids'],
    q_seqs['input_ids'], 
    q_seqs['attention_mask'], 
    # q_seqs['token_type_ids']
    )
valid_loader = DataLoader(
    valid_dataset,
    batch_size=model_args.batch_size
    )


## Import from Fine_Tune_BERT_on_SQuAD_v1_1.ipynb

In [16]:
# look inside dataset at huggingface
vars(train_dataset_from_huggingface)

#  title: string
#  context: string
#  question: string
#  id: string
#  answers: struct < answer_start: list<item: int64>, text: list<item: string> >
#    child 0, answer_start: list<item: int64>
#        child 0, item: int64
#    child 1, text: list<item: string>
#        child 0, item: string
#  document_id: int64

{'_info': DatasetInfo(description='', citation='', homepage='', license='', features={'__index_level_0__': Value(dtype='int64', id=None), 'answers': {'answer_start': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), 'text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}, 'context': Value(dtype='string', id=None), 'document_id': Value(dtype='int64', id=None), 'id': Value(dtype='string', id=None), 'question': Value(dtype='string', id=None), 'title': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None),
 '_split': None,
 '_indexes': {},
 '_data': pyarrow.Table
 title: string
 context: string
 question: string
 id: string
 answers: struct<answer_start: list<item: int64>, text: list<item: string>>
   child 0, answer_start: list<item: int64>
       child 0

In [21]:
train_dataset_from_huggingface[1]

{'title': '인사조직관리',
 'context': "'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 1950년대이다. 2차 세계대전을 마치고, 6.25전쟁의 시기로 유럽은 전후 재건에 집중하고, 유럽 제국주의의 식민지가 독립하여 아프리카, 아시아, 아메리카 대륙에서 신생국가가 형성되는 시기였고, 미국은 전쟁 이후 경제적 변화에 기업이 적응을 해야 하던 시기였다. 특히 1954년 피터 드러커의 저서 《경영의 실제》는 현대적 경영의 기준을 제시하여서, 기존 근대적 인사조직관리를 넘어선 현대적 인사조직관리의 전환점이 된다. 드러커는 경영자의 역할을 강조하며 경영이 현시대 최고의 예술이자 과학이라고 주장하였고 , 이 주장은 21세기 인사조직관리의 역할을 자리매김했다.\\n\\n현대적 인사조직관리와 근대 인사조직관리의 가장 큰 차이는 통합이다. 19세기의 영향을 받던 근대적 경영학(고전적 경영)의 흐름은 기능을 강조하였지만, 1950년대 이후의 현대 경영학은 통합을 강조하였다. 기능이 분화된 '기계적인 기업조직' 이해에서 다양한 기능을 인사조직관리의 목적, 경영의 목적을 위해서 다양한 분야를 통합하여 '유기적 기업 조직' 이해로 전환되었다. 이 통합적 접근방식은 과정, 시스템, 상황을 중심으로 하는 인사조직관리 방식을 형성했다.",
 'question': '현대적 인사조직관리의 시발점이 된 책은?',
 'id': 'mrc-0-004397',
 'answers': {'answer_start': [212], 'text': ['《경영의 실제》']},
 'document_id': 51638,
 '__index_level_0__': 2873}

## T5! T5!

![img](https://pbs.twimg.com/media/D2RWVuzWoAEw8fm?format=jpg&name=medium)

In [25]:
!git clone git@github.com:AIRC-KETI/ke-t5.git
%cd ke-t5
!pip3 install -r requirements.txt
%cd ..

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Cloning into 'ke-t5'...
remote: Enumerating objects: 197, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (172/172), done.
remote: Total 197 (delta 111), reused 85 (delta 13), pack-reused 0
Receiving objects: 100% (197/197), 783.74 KiB | 1.08 MiB/s, done.
Resolving deltas: 100% (111/111), done.
/opt/ml/mrc-level2-nlp-15/ke-t5
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM

In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# https://github.com/AIRC-KETI/ke-t5
# downstream tasks: ke_t5_korquad_allanswers, klue_mrc_gen, ke_t5_ko_en_qa_equal, ke_t5_trivi_qa_v010
# ke_t5_ko_en_qa_proportional, ke_t5_ko_en_qa_equal, ke_t5_korquad_allanswers, ke_t5_squad_v010_allanswers, ke_t5_trivia_qa_v010
# Summarization과 Extractive QA의 경우 입력이 모델들의 최대 시퀀스 길이인 512를 넘을 수가 있습니다. 이 경우 넘는 부분을 그냥 버리고 학습과 성능측정을 진행했습니다. 즉, 요약의 경우 512 토큰을 초과하는 부분은 요약이 되지 않고, extractive QA의 경우 question에 대한 정답이 512 토큰을 초과하는 부분에 존재하면 답을 찾을 수 없습니다. 이러한 경우를 처리하여 성능을 높이기 위해서는 직접 학습 프로그램을 만들어 사용하셔야 합니다. (e.g. Extractive QA의 경우, BERT에서처럼 document span으로 분리하여 해당 span에 정답이 있는지 없는지로 학습.
model_name = 'KETI-AIR/ke-t5-base'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)